<a href="https://colab.research.google.com/github/hoomanabasi/BERT-Sentiment-Analysis/blob/main/BERT%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from datasets import load_dataset
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer
from datasets import Dataset
from transformers import TFBertForSequenceClassification

In [8]:
dataset = load_dataset("carblacac/twitter-sentiment-analysis")
def clean_text(text):
    text = text.lower()  # تبدیل به حروف کوچک
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # حذف لینک‌ها
    text = re.sub(r'\@\w+|\#', '', text)  # حذف تگ‌ها و هشتگ‌ها
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # حذف کاراکترهای غیرضروری
    text = re.sub(r'\s+', ' ', text).strip()  # حذف فاصله‌های اضافی
    return text
dataset=dataset.map(lambda x:{"text":clean_text(x["text"])})
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")
def tokenizer_def(example):
    return tokenizer(example["text"], padding="max_length", truncation=True,max_length=128)

tokenized_datasets = dataset.map(tokenizer_def, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.44k [00:00<?, ?B/s]

twitter-sentiment-analysis.py:   0%|          | 0.00/4.38k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

The repository for carblacac/twitter-sentiment-analysis contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/carblacac/twitter-sentiment-analysis.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/119988 [00:00<?, ? examples/s]

Map:   0%|          | 0/29997 [00:00<?, ? examples/s]

Map:   0%|          | 0/61998 [00:00<?, ? examples/s]

In [9]:
print(tokenized_datasets["train"].column_names)

['text', 'feeling', 'input_ids', 'token_type_ids', 'attention_mask']


In [10]:
def prepare_tf_dataset(dataset, shuffle=True, batch_size=32):
    tf_dataset = dataset.to_tf_dataset(
        columns=["input_ids", "attention_mask", "token_type_ids"],
        label_cols=["feeling"],
        shuffle=shuffle,
        batch_size=batch_size,
        drop_remainder=False
    )
    return tf_dataset

train_tf_dataset = prepare_tf_dataset(tokenized_datasets["train"])
test_tf_dataset = prepare_tf_dataset(tokenized_datasets["test"])


/usr/local/lib/python3.11/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [11]:
model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
opt=tf.keras.optimizers.Adam(learning_rate=5e-5)
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric=tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=opt,loss=loss,metrics=[metric])
hist=model.fit(train_tf_dataset,validation_data=test_tf_dataset,epochs=4,verbose=1)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/4
 292/3750 [=>............................] - ETA: 44:21 - loss: 0.4778 - accuracy: 0.7764

KeyboardInterrupt: 